In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bootleg_data_prep.benchmarks.filter_and_compute_recall as fcr
import bootleg_data_prep.benchmarks.candidate_generators as gen
import bootleg_data_prep.utils.data_prep_utils as prep_utils
import marisa_trie
import unicodedata
from collections import defaultdict

import json
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from tqdm import tqdm

ps = PorterStemmer()
nlp = spacy.load("en_core_web_sm")
stopwords = set(stopwords.words('english'))

import bootleg_data_prep.utils.data_prep_utils as prep_utils
from bootleg.utils import utils
from bootleg.symbols.entity_symbols import EntitySymbols
import glob, os, jsonlines, json, time
from tqdm import tqdm
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [6]:
import sys
sys.path.append('.')
from utils_not_for_public import load_esp

In [10]:
es, esp, type_vocab, typeid2typename, type_vocab_wd, typeid2typename_wd, alias_qid_count = load_esp("wiki_0905", overwrite=True) 

Removing cache _cache_wiki0905...
Error trying to load _cache_wiki0905/entity_symbols_plus.pkl. The file may not exist. Regenerating...
Entity save directory where all information is saved/loaded from is /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings
Found existing qid_count at /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_qid_count
Reading types from /dfs/scratch0/lorr1/data_prep/embs with type file hyena_types_0905.json...
Loading types from /dfs/scratch0/lorr1/data_prep/embs/hyena_types_0905.json


Reading /dfs/scratch0/lorr1/data_prep/embs/hyena_types_0905.json: 100%|██████████| 5310039/5310039 [00:33<00:00, 159136.28it/s]


Reading types from /dfs/scratch0/lorr1/data_prep/embs with type file wikidata_types_0905.json...
Loading types from /dfs/scratch0/lorr1/data_prep/embs/wikidata_types_0905.json


Reading /dfs/scratch0/lorr1/data_prep/embs/wikidata_types_0905.json: 100%|██████████| 5310039/5310039 [00:31<00:00, 170013.49it/s]


5310039 hyena qids have a type 5310039 wd qids have a type
Reading kg adj from /dfs/scratch0/lorr1/data_prep/embs with kg file kg_adj_0905.txt...
Reading kg_adj from /dfs/scratch0/lorr1/data_prep/embs/kg_adj_0905.txt...


...

Reading in existing affordance bags from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_type_words
0 for affordance_types
Reading in existing entity word bags from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_bag_of_words
0 for entity_words
Reading in vocab from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_all_words/all_words_vocab.marisa
Error reading vocab marisa-trie/lib/marisa/grimoire/io/mapper.cc:141: MARISA_IO_ERROR: ::stat(filename, &st) != 0
TYPE QID LENGTHS WIKIDATA 5310039 HYENA 5310039
Max Values {'alias2qid': 30, 'rel_mapping': 150, 'qid2typeid_hy': 3, 'qid2typeid_wd': 3}
FOUND TRI NAME alias2qid
FOUND TRI NAME rel_mapping
FOUND TRI NAME qid2typeid_hy
FOUND TRI NAME qid2typeid_wd
Finished building entity symbols
Reading in vocab from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_all_words/all_words_vocab.marisa
Error reading vocab marisa-trie/lib/mar

Reading /dfs/scratch0/lorr1/data_prep/embs/hyena_types_0905.json: 100%|██████████| 5310039/5310039 [00:33<00:00, 158578.94it/s]


Loading types from /dfs/scratch0/lorr1/data_prep/embs/wikidata_types_0905.json


Reading /dfs/scratch0/lorr1/data_prep/embs/wikidata_types_0905.json: 100%|██████████| 5310039/5310039 [00:32<00:00, 164824.26it/s]


Loading alias qid counts from /dfs/scratch0/lorr1/data_prep/data/wiki0905/stats/alias_qid_traindata_withaugment.json
Found 506 hyena types
Found 26933 wikidata types
FINISHED LOADING IN 2598.595745563507


In [3]:
from argparse import Namespace

args = Namespace(
    data_dir = '/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest',
    entity_dump = '/dfs/scratch0/lorr1/projects/bootleg/data/wiki_0906/entity_db/entity_mappings',
    sub_dir = 'unfiltered',
    file = "",
    out_dir = 'filtered',
    errors_dir = 'cand_gen_filtering_errors',
    verbose = True,
    method = "contextual",
    expand_aliases = True,
    gold_given = True,
    max_candidates = 30,
    large_alias_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids.json',
    large_title_map = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/qid2title.json',
    wiki_pages = '/dfs/scratch0/lorr1/data_prep/unfiltered_data_0914/'
)

In [4]:
def load_wikipedia_pages(args):
    files = sorted(glob.glob(os.path.join(args.wiki_pages, '*.jsonl')), reverse=True)
    print(f"Loading {len(files)} wikipedia pages...")
    st = time.time()
    qid2page = {}
    for file in tqdm(files):
        with jsonlines.open(file) as in_file:
            for obj in in_file:
                qid2page[obj['qid']] = obj
    print(f"Took {time.time()-st}s to load {len(files)} files")
    return qid2page

def load_large_alias_map(args, entity_dump):
    temp_dir = "_temp"
    utils.ensure_dir(temp_dir)
    temp_f = os.path.join(temp_dir, "alias_map.pkl")
    if os.path.exists(temp_f):
        print(f"Reading alias map from {temp_f}")
        filtered_alias_map = utils.load_pickle_file(temp_f)
        return filtered_alias_map
    large_alias_to_qid_map = json.load(open(args.large_alias_map))

    # filter down and remove qids which aren't contained in the entity save
    filtered_alias_map = {}
    for alias, qid_list in large_alias_to_qid_map.items():
        filtered_qid_list = []
        for qid, count in qid_list:
            if entity_dump.qid_exists(qid):
                filtered_qid_list.append([qid, count])
        filtered_alias_map[alias] = filtered_qid_list
    utils.dump_pickle_file(temp_f, filtered_alias_map)
    return filtered_alias_map

def load_large_title_map(args):
    return json.load(open(args.large_title_map))

def write_output_file(sentences, fpath, args):
    print(f"Output to {fpath}")
    out_file = open(fpath, 'w')
    for i, sentence in enumerate(sentences):
        out_file.write(json.dumps(sentence) + "\n")

In [5]:
qid2page = load_wikipedia_pages(args)

  0%|          | 0/241 [00:00<?, ?it/s]

Loading 241 wikipedia pages...


100%|██████████| 241/241 [31:05<00:00,  7.74s/it]  

Took 1865.771460533142s to load 241 files


In [6]:
entity_dump = EntitySymbols(load_dir=args.entity_dump)
alias_map = load_large_alias_map(args, entity_dump)
title_map = load_large_title_map(args)
alias_tri = marisa_trie.Trie(alias_map.keys())

Reading alias map from _temp/alias_map.pkl


In [7]:
class Contextual:

    def __init__(self, args, entity_dump, qid2page, alias_map, alias_tri, title_map):

        self.args = args
        self.entity_dump = entity_dump
        self.filtered_alias_map = alias_map
        self.title_map = title_map
        self.alias_tri = alias_tri
        self.qid2page = qid2page
        self.new_alias2qid = {}
        self.saved_metrics = defaultdict(int)
        self.detailed_metrics = defaultdict(list)

    def convert_char_to_word_boundaries(self, phrase, start_char, end_char):
        start_word = max(0, len(phrase[:start_char].split()))
        end_word = max(0, len(phrase[:end_char].split()))
        return start_word, end_word

    def get_lnrm(self, s):
        """Convert a string to its lnrm form
        We form the lower-cased normalized version l(s) of a string s by canonicalizing
        its UTF-8 characters, eliminating diacritics, lower-casing the UTF-8 and
        throwing out all ASCII-range characters that are not alpha-numeric.
        from http://nlp.stanford.edu/pubs/subctackbp.pdf Section 2.3
        Args:
            input string
        Returns:
            the lnrm form of the string
        """
        lnrm = unicodedata.normalize('NFD', str(s))
        lnrm = lnrm.lower()
        lnrm = ''.join([x for x in lnrm if (not unicodedata.combining(x)
                                            and x.isalnum() or x == ' ')]).strip()
        # will remove if there are any duplicate white spaces e.g. "the  alias    is here"
        lnrm = " ".join(lnrm.split())
        return lnrm

    def get_proper_nouns(self, text):
        words = text.split()
        tagged_text = pos_tag(words)
        # all_nouns = [word for word,pos in tagged_text if pos[:2] in ['NN'] and (word.lower() not in stopwords) and (word.lower().rstrip("s") not in stopwords)]
        all_nouns = []
        #
        # doc = nlp(text)
        # ent_idx = 0
        # while ent_idx < len(doc.ents):
        #     ent = doc.ents[ent_idx]
        #     ent_text = ent.text
        #     ent_idx += 1
        #     print("ENT TX", ent_text)
        i = 0
        nouns = []
        while i < len(words):
            if words[i][0].isupper() and words[i].lower() not in stopwords:
                start_i = i
                i += 1
                while True and i < len(words):
                    if words[i][0].isupper():
                        i += 1
                    else:
                        break
                nouns.append(' '.join(words[start_i: i]))
                continue
            i += 1
        full_nouns = set(all_nouns)
        for noun in nouns:
            full_nouns.add(noun)
            for part in noun.split():
                full_nouns.add(part)
        return list(full_nouns)

    def get_proper_nouns_old(self, text):
        nouns = []
        words = text.split()
        start = 0
        i = 0
        while i < len(words):
            if words[i][0].isupper():
                start_i = i
                i += 1
                while True and i < len(words):
                    if words[i][0].isupper():
                        i += 1
                    else:
                        break
                nouns.append(' '.join(words[start_i: i]))
                continue
            i += 1
        full_nouns = []
        for noun in nouns:
            full_nouns.append(noun)
            for part in noun.split():
                full_nouns.append(part)
        return list(set(full_nouns))

    def get_sub_alias_match(self, alias, alias2qids):
        assert len(alias.strip()) > 0, f"The alias is empty. This shouldn't happen."
        words = alias.split()
        subseqs = set()
        for i in range(len(words)):
            for j in range(i, len(words)+1):
                phrase = ' '.join(words[i:j])
                if len(phrase) > 0:
                    subseqs.add(phrase)
        subseqs = list(subseqs)
        subseqs.sort(key=lambda s: len(s), reverse=True)
        qid2count = defaultdict(int)
        curr_len = len(subseqs[0].split())
        for alias in subseqs:
            if len(alias.split()) != curr_len:
                if len(qid2count) > 0:
                    break
                curr_len = len(alias.split())
            if alias in alias2qids:
                for pair in alias2qids[alias]:
                    qid2count[pair[0]] = pair[1] + qid2count[pair[0]]
        candidates = [[qid, value] for qid, value in qid2count.items() if value > 0]
        return candidates

    def get_sub_alias_match2(self, alias, alias2qids):
        assert len(alias.strip()) > 0, f"The alias is empty. This shouldn't happen."
        words = alias.split()
        subseqs = set()
        for i in range(len(words)):
            for j in range(i, len(words)+1):
                phrase = ' '.join(words[i:j])
                if len(phrase) > 0 and len(phrase.split()) > 0:
                    subseqs.add(phrase)
        if len(subseqs) <= 0:
            return []
        subseqs = list(subseqs)
        subseqs.sort(key=lambda s: len(s), reverse=True)
        qid2count = defaultdict(int)
        curr_len = len(subseqs[0].split())
        for alias in subseqs:
            if len(alias.split()) != curr_len:
                if len(qid2count) > 0:
                    break
                curr_len = len(alias.split())
            if alias in alias2qids:
                for pair in alias2qids[alias]:
                    qid2count[pair[0]] = pair[1] + qid2count[pair[0]]
        candidates = [[qid, value] for qid, value in qid2count.items() if value > 0]
        return candidates

    def get_qids(self, target_alias, alias2qids):       
        if target_alias in alias2qids:
            return alias2qids[target_alias]

        if target_alias.replace(" ", "") in alias2qids:
            return alias2qids[target_alias.replace(" ", "")]

        return self.get_sub_alias_match2(target_alias, alias2qids)


    def recompute_qid_rank(self, sample, alias, true_qid, qid2page, alias2qid, entity_dump):
        qid_pairs = self.get_qids(alias, alias2qid)
        if len(qid_pairs) == 0:
            self.saved_metrics["no_alias"] += 1
            self.detailed_metrics["no_alias"].append(
                {
                    "true_qid": true_qid,
                    "alias": alias
                }
            )
        if len(qid_pairs) <= 1:
            self.saved_metrics["len_1_alias"] += 1
            return qid_pairs
        # Each element in qid_pairs is [qid, qid_score]
        qid_pairs.sort(key=lambda x:x[1], reverse=True)
        context = sample['sentence']
        # Extract proper (capitalized) nouns from sentence
        prop_nouns = [word.lower() for word in self.get_proper_nouns(context) if word.lower() != alias]
        qids = [x[0] for x in qid_pairs]
        saved_counts = {}
        # Count number of times proper noun appears on Wikipedia page
        for i, qid in enumerate(qids):
            count = {p: 0 for p in prop_nouns}
            if qid in qid2page:
                for sentence in qid2page[qid]['sentences']:
                    text = " " + sentence['sentence'].lower() + " "
                    for term in prop_nouns:
                        if f" {term} " in text:
                            count[term] += 1
            saved_counts[qid] = count
        counts = [x[1] for x in qid_pairs]
        # Compute score for qid based on counts of proper nouns
        for i, qid in enumerate(qids):
            score = 0
            num_hits = 0
            for w in list(saved_counts[qid].keys()):
                x = saved_counts[qid][w]
                if x > 0:
                    num_hits += 1
                    # Heuristic threshold function
                    score += min(x, 5)*len(w.split())
            counts[i] += 10000*score
        # TODO: fix if acronym
        # SCORE BASED ON ALIAS TO ENTITY TITLE (partial)
        for i, qid in enumerate(qids):
            title = self.get_lnrm(entity_dump.get_title(qid).lower())
            title_score = fuzz.partial_ratio(title, alias)/100
            counts[i] += 100000*title_score
        # ADD TYPE SCORE

        scored_qids = [list(x) for x in zip(qids, counts)]
        scored_qids = sorted(scored_qids, key= lambda x: x[1], reverse = True)
        # Take top 5 most popular qids and remaining 25 from scored qids
        final_scored_qids = qid_pairs[:5]
        only_qids = [x[0] for x in final_scored_qids]
        for qid_pair in scored_qids:
            if len(final_scored_qids) == self.args.max_candidates:
                break
            if qid_pair[0] not in only_qids:
                only_qids.append(qid_pair[0])
                final_scored_qids.append(qid_pair)

        if self.args.gold_given:
            if true_qid not in only_qids[:self.args.max_candidates]:
                if true_qid in [x[0] for x in scored_qids]:
                    self.saved_metrics["in_long_cands_not_short"] += 1
                    self.detailed_metrics["in_long_cands_not_sort"].append(
                        {
                            "true_qid": true_qid,
                            "alias": alias,
                            "context": context,
                            "nouns": prop_nouns,
                            "saved_counts": saved_counts,
                            "scored_qids": scored_qids,
                            "titles": [entity_dump.get_title(q) for q in only_qids]
                        }
                    )
#                     print("TRUE QID", true_qid, entity_dump.get_title(true_qid))
#                     print("CONTEXT", context)
#                     print("NOUNS", prop_nouns)
#                     print("COUNTS", saved_counts[true_qid])
#                     for tqid in only_qids[:self.args.max_candidates]:
#                         print("OTHER COUNTS", tqid, entity_dump.get_title(tqid),  saved_counts[tqid])
#                     print("SCORES", scored_qids[:50])
#                     print("TITLES", [entity_dump.get_title(q) for q in only_qids[:50]])
                    # print("ALL SENTENCES", qid2page[true_qid]['sentences'])
                else:
                    self.saved_metrics["not_long_cands_not_short"] += 1
                    self.detailed_metrics["not_long_cands_not_sort"].append(
                        {
                            "true_qid": true_qid,
                            "alias": alias,
                            "context": context,
                            "nouns": prop_nouns,
                            "saved_counts": saved_counts,
                            "scored_qids": scored_qids,
                            "titles": [entity_dump.get_title(q) for q in only_qids]
                        }
                    )
            else:
                self.saved_metrics["success"] += 1
        return final_scored_qids[:self.args.max_candidates]

    def filter_sentence(self, sentence):
        success = 0
        no_alias_exists = 0
        qid_not_in_cands = 0
        no_qid_in_dump = 0
        errors = []
        new_sentence = {
            'sentence': sentence['sentence'],
            'sent_idx_unq': sentence['sent_idx_unq'],
            'aliases': [],
            'spans': [],
            'qids': [],
            'gold': [],
            'cands': []
        }
        phrase = sentence['sentence']
        aliases = sentence['aliases']
        spans = sentence['spans']
        # Uses NER tagger to expand aliases
        if self.args.expand_aliases:
            doc = nlp(phrase)
            al_idx = 0
            ent_idx = 0
            while ent_idx < len(doc.ents) and al_idx < len(aliases):
                ent = doc.ents[ent_idx]
                ent_text = ent.text
                ent_start_char = ent.start_char
                ent_end_char = ent.end_char
                # remove starting "the" and trailing "'s"
                if ent_text.startswith("the "):
                    ent_text = ent_text[4:]
                    ent_start_char += 4
                if ent_text.endswith(" 's"):
                    ent_text = ent_text[:-3]
                    ent_end_char -= 3
                start_span, end_span = self.convert_char_to_word_boundaries(phrase, ent_start_char, ent_end_char)
                cur_alias = aliases[al_idx]
                cur_spans = spans[al_idx]
                is_bleeding_into_next = False
                if al_idx+1 < len(spans):
                    next_cur_start_span = spans[al_idx+1][0]
                    is_bleeding_into_next = end_span >= next_cur_start_span
                if start_span >= cur_spans[1]:
                    al_idx += 1
                elif start_span <= cur_spans[0] and end_span >= cur_spans[1] and not is_bleeding_into_next and aliases[al_idx] != ent_text.lower():
#                     print("Swapping", start_span, end_span, cur_spans, cur_alias, "**", ent_text, "**", phrase)
                    spans[al_idx] = [start_span, end_span]
                    aliases[al_idx] = ent_text
                    al_idx += 1
                    ent_idx += 1
                else:
                    ent_idx += 1
        if self.args.gold_given:
            qids = sentence['qids']
        else:
            qids = ["Q-1" for _ in range(len(sentence['aliases']))]
        gold = sentence.get('gold', [True for _ in range(len(sentence['aliases']))])
        sent_idx_unq = sentence['sent_idx_unq']
        for i in range(len(aliases)):
            alias = prep_utils.get_lnrm(aliases[i])
            candidates = self.recompute_qid_rank(sentence, alias, qids[i], self.qid2page, self.filtered_alias_map, self.entity_dump)
            #######
            new_sentence["cands"].append(candidates)
            # With no golds and not nil model, need to set QIDs to be something in the save; nil models can have Q-1
            qid_to_add = qids[i]
#             if not self.args.gold_given and len(candidates) > 0:
#                 qid_to_add = candidates[0][0]
            #print(f"Alias: {alias}, Candidates {candidates}")
            if len(candidates) == 0:
                # print(f"{alias} not in alias map. True QID: {qids[i]}")
                errors.append({
                    'alias': alias,
                    'qid': qid_to_add,
                    'sentence': sentence['sentence'],
                    'label': 'alias not found',
                })
                no_alias_exists += 1
                continue
            if self.args.gold_given and not self.entity_dump.qid_exists(qid_to_add):
                print(f"QID {qid_to_add} doesn't exist in entity save for {alias}.")
                no_qid_in_dump += 1
                continue
            if qid_to_add in [x[0] for x in candidates]:
                assert int(spans[i][1]) <= len(sentence['sentence'].split()), sentence
                alias = f"{alias}_{sent_idx_unq}"
                self.new_alias2qid[alias] = candidates
                new_sentence['aliases'].append(alias)
                new_sentence['qids'].append(qid_to_add)
                new_sentence['spans'].append(spans[i])
                new_sentence['gold'].append(gold[i])
                success += 1
            else:
                # print(f"Alias: {alias}, True QID: {qids[i]}, Candidates in map: {candidates}")
                errors.append({
                    'alias': alias,
                    'candidates': list(candidates),
                    'qid': qid_to_add,
                    'sentence': sentence['sentence'],
                    'label': 'candidate not found',
                })
                assert int(spans[i][1]) <= len(sentence['sentence'].split()), sentence
                alias = f"{alias}_{sent_idx_unq}"
                self.new_alias2qid[alias] = candidates
                new_sentence['aliases'].append(alias)
                new_sentence['qids'].append(qid_to_add)
                new_sentence['spans'].append(spans[i])
                new_sentence['gold'].append(gold[i])
                qid_not_in_cands += 1

        # return number of aliases kept, and dropped
        return new_sentence, success, no_alias_exists, qid_not_in_cands, no_qid_in_dump, errors

In [8]:
out_dir = prep_utils.get_outdir(args.data_dir, args.out_dir)
errors_dir = prep_utils.get_outdir(args.data_dir, args.errors_dir)

print(f"Candidate generator: {args.method}")
if args.method == 'standard':
    candidate_generator = gen.Standard(args, entity_dump)
elif args.method == 'contextual':
    candidate_generator = Contextual(args, entity_dump, qid2page, alias_map, alias_tri, title_map)

Making /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/filtered
Making /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/cand_gen_filtering_errors
Candidate generator: contextual


In [9]:
candidate_generator.saved_metrics = defaultdict(int)
candidate_generator.detailed_metrics = defaultdict(list)
if args.file:
    files = [args.file]
else:
    files = prep_utils.glob_files(os.path.join(args.data_dir, args.sub_dir, '*.jsonl'))
recall_results = {}
for f in files:
    print(f"Filtering file {f}")
    fname = os.path.basename(f)
    if fname == "test_rss500_clean.jsonl":
        print(f"Skipping {fname}")
        continue
    filtered_sentences, errors, recall = fcr.filter_and_clean(f, candidate_generator, args)
    write_output_file(filtered_sentences, os.path.join(out_dir, fname), args)
    fcr.write_errors(errors, os.path.join(errors_dir, fname.replace('jsonl', 'txt')))
    recall_results[fname] = recall

entity_dump_dir = os.path.join(out_dir, 'entity_db/entity_mappings')
utils.ensure_dir(entity_dump_dir)
new_alias_f = os.path.join(entity_dump_dir, "alias2qid_new.json")
# save new alias mapping
with open(new_alias_f, "w") as out_f:
    json.save(candidate_generator.new_alias2qid, out_f)
print(f"Saved new alias mapping to {new_alias_f}")

result_fpath = os.path.join(errors_dir, 'recall_results.json')
json.save(recall_results, open(result_fpath, 'w'))
json.save(candidate_generator.detailed_metrics, open(os.path.join(errors_dir, "detailed.json"), "w"))
print(f"Recall {recall_results}")
print(f"Metrics {json.dumps(candidate_generator.saved_metrics, indent=4)}")

Processing sentences:   1%|          | 2/357 [00:00<00:18, 19.34it/s]

Filtering file /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/unfiltered/test_rss500.jsonl


Processing sentences:  17%|█▋        | 61/357 [00:06<00:39,  7.51it/s]

QID -1 doesn't exist in entity save for mohr davidow ventures.


Processing sentences:  40%|████      | 143/357 [00:13<00:16, 12.60it/s]

...

QID Q3453515 doesn't exist in entity save for russia in global affairs.


Processing sentences:  98%|█████████▊| 351/357 [00:35<00:00, 20.08it/s]

QID Q6979295 doesn't exist in entity save for national venture capital association.


Processing sentences: 100%|██████████| 357/357 [00:36<00:00,  9.90it/s]


/dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/unfiltered/test_rss500.jsonl. Total mentions 524. Succesfully mapped: 473. Alias not found for 1 mentions. Correct QID not in candidate list for 46 mentions. Correct QID not in save for 4 mentions.
Output to /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/filtered/test_rss500.jsonl
Filtering file /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/unfiltered/test_rss500_clean.jsonl
Skipping test_rss500_clean.jsonl
Saved new alias mapping to /dfs/scratch0/lorr1/projects/bootleg/data/benchmarks/rss500wikinewdatatest/filtered/entity_db/entity_mappings/alias2qid_new.json
Recall {'test_rss500.jsonl': {'total_mentions': 524, 'found_mentions': 473, 'alias_in_list': 523}}
Metrics {
    "len_1_alias": 193,
    "success": 288,
    "not_long_cands_not_short": 33,
    "in_long_cands_not_short": 10,
    "no_alias": 1
}


## Debug

In [11]:
q = "Q938005"

In [ ]:
# To load alias2qids,

a2q_modified = json.load(open("/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids.json"))
q2title = json.load(open("/dfs/scratch0/lorr1/projects/bootleg/data/wiki_0906/entity_db/entity_mappings/qid2title.json"))
q2score = {}

for al in a2q_modified:
    for pair in a2q_modified[al]:
        qid, score = pair
        if qid in q2score:
            assert score == q2score[qid]
            continue
        else:
            q2score[qid] = score


person_set, gender_map = np.load('/dfs/scratch0/lorr1/projects/bootleg/data/wikidata_mappings/person.npy', allow_pickle=True)

# for each person: modify the alias
for qid in q2title:
    if qid is person:
        name = q2title[q]
        new_alias = get_lnrm(strip_middle(name))
        if new_alias in a2q_modified:
            a2q_modified[new_alias].append([qid, q2score[qid]])
        else:
            a2q_modified[new_alias] = [[qid, q2score[qid]]]

json.save(a2q_modified, open("/dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids_modified.json"))
# In the args above, use /dfs/scratch0/lorr1/data_prep/unfiltered_data_0906/entity_db/entity_mappings/alias2qids_modified.json for large alias map parameter